In [ ]:
# Importing all required libraries for environment management, data processing, model integration, and API use.
import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI

In [ ]:
# Load Hugging Face access token from environment and authenticate to the Hugging Face Hub.

load_dotenv(override=True)
os.environ['HUGGINGFACE_RW_TOKEN'] = os.getenv('HUGGINGFACE_RW_TOKEN', 'your-key-if-not-using-env')

hf_token = os.environ['HUGGINGFACE_RW_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# Load a subset of the scientific papers dataset (train and test) from Hugging Face Datasets.

from datasets import load_dataset

dataset = load_dataset("scientific_papers", "arxiv", split="train[:1000]", trust_remote_code=True)
test = load_dataset("scientific_papers", "arxiv", split="test[:10]", trust_remote_code=True)
print(test[0]['article'])


In [ ]:
# Define a function to clean and normalize scientific text using regex patterns.

import re

def clean_text(text):
    text = re.sub(r'@\w+', '', text)

    text = re.sub(r'\$+', '', text)
    text = re.sub(r'\\[a-zA-Z]+\b', '', text) 

    text = re.sub(r'\[[^\]]{1,30}\]', '', text)  

    text = re.sub(r'[\*\?·‥…‧]+', '', text)

    text = re.sub(r'[,.]{2,}', '.', text)

    text = re.sub(r'\s+', ' ', text)

    return text.strip()



In [ ]:
# Apply the cleaning function to all samples in the dataset and store cleaned articles and abstracts.

from tqdm import tqdm
cleaned_dataset = []
for sample in tqdm(dataset): 
    cleaned_sample = {
        "article": clean_text(sample["article"]),
        "abstract": clean_text(sample["abstract"])
    }
    cleaned_dataset.append(cleaned_sample)

In [ ]:
# Generate instruction-style prompt-completion pairs from cleaned dataset and save to JSONL for fine-tuning.

import random
output_dir = "data"
output_path = os.path.join(output_dir, "scipaper_instruct_100.jsonl")

os.makedirs(output_dir, exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    for i in tqdm(range(100)):
        article = cleaned_dataset[i]['article']
        abstract = cleaned_dataset[i]['abstract']



        # summary
        prompt = f"""Instruction:
Summarize the following article.

Article:
{article}

### Response:"""
        completion = f" {abstract.strip()}" 
        
        f.write(json.dumps({
            "prompt": prompt,
            "completion": completion
        }) + "\n")

print(f"Saved to {output_path}")

In [ ]:
# Preview the first few entries in the generated JSONL file to verify formatting and content.

with open("data/scipaper_instruct_1000.jsonl", "r", encoding="utf-8") as f:
    for i in range(3):
        print(json.loads(f.readline()))

In [ ]:
# Define model names, user ID, and run name with timestamp for logging and saving to Hugging Face hub.

from datetime import datetime
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "ScientificPaperLLMs"
HF_USER = "Benny97"
# Run name for saving the model in the hub
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

QUANT_4_BIT = True

In [ ]:
# Import necessary libraries for dataset loading, model configuration, and PEFT training.

import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig,set_seed
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# Configure quantization settings (4-bit or 8-bit) based on QUANT_4_BIT flag for efficient model loading.

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# Load tokenizer and base model with quantization, and report memory usage after loading the model.

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map='cuda'
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
# Define the data collator to identify the response section during training using the response template.

from trl import DataCollatorForCompletionOnlyLM
collator = DataCollatorForCompletionOnlyLM(response_template="### Response:", tokenizer=tokenizer)

In [ ]:
# Load the dataset and filter out samples whose tokenized prompt + completion exceed the max token limit.

max_len = 3000
data_path = "data/scipaper_instruct_1000.jsonl"
dataset = load_dataset("json", data_files=data_path)["train"]
def filter_fn(example):
    merged = example["prompt"] + example["completion"]
    return len(tokenizer(merged)["input_ids"]) <= max_len

filtered_dataset = dataset.filter(filter_fn)


In [ ]:
# Configure LoRA, training hyperparameters, and initialize the SFTTrainer for fine-tuning the model.

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)


training_args = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=50,
    num_train_epochs=1,
    learning_rate=1e-4,
    lr_scheduler_type='cosine',
    fp16=False,
    bf16=True,
    save_total_limit=2,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    report_to="none",
    max_seq_length=12000,         
    gradient_checkpointing=True,
    dataset_text_field="text",
    group_by_length=True,
)



fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=filtered_dataset,
    peft_config=lora_config,
    args=training_args,
    data_collator=collator,
    
  )



In [ ]:
# Start the fine-tuning process using the configured model and training arguments.

fine_tuning.train()

In [ ]:
# Upload the fine-tuned adapter model to your Hugging Face Hub repository.

fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}") 

In [ ]:
# Define naming variables for reloading the uploaded fine-tuned model.

RUN_NAME = "2025-05-10_22.43.03"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

In [ ]:
# Load the fine-tuned adapter model from the Hugging Face Hub into memory.

from peft import PeftModel
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
# Generate model output based on a prompt using the fine-tuned model.

def model_Gen(prompt):
    set_seed(42)

    if "### Response:" in prompt:
        prompt = prompt.split("### Response:")[0] + "### Response:"


    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = fine_tuned_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=False,
        num_return_sequences=1,
    )

    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return response.strip()

In [ ]:
# Create a formatted prompt (summary or Q&A) and call the model to generate a response.

def generate_response(article, user_message, mode):
    if not article:
        return "Please upload or input an article."

    if mode == "Summarize":
        prompt = f"""### Instruction:
                    Summarize the following article.
                    
                    ### Article:
                    {article}
                    
                    ### Response:"""
    else:  # Chat mode
        if not user_message:
            return "Please enter a message to chat about the article."
        prompt = f"""### Instruction:
                    You are a helpful assistant. Answer the question below based on the article.
                    
                    ### Article:
                    {article}
                    
                    ### Question:
                    {user_message}
                    
                    ### Response:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(fine_tuned_model.device)
    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Response:" in generated:
        return generated.split("### Response:")[-1].strip()
    return generated.strip()

In [ ]:
# Import Gradio library for building the web UI.

import gradio as gr

In [ ]:
# Build the Gradio app with upload, text input, and generate response buttons.

def process_file(file):
    with open(file.name, "r", encoding="utf-8") as f:
        content = f.read()
    return content
def clear_article():
    return ""
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Scientific Article Assistant")
    mode = gr.Radio(["Summarize", "Chat"], value="Summarize", label="Mode")
    article_input = gr.Textbox(lines=15, label="Paste Article or Upload File")
    file_upload = gr.File(label="Or Upload a .txt File", file_types=[".txt"])
    clear_btn = gr.Button("🗑 Clear Article Input", variant="secondary")
    user_message = gr.Textbox(lines=2, label="Your Question (for Chat mode)")
    output = gr.Textbox(label="Model Response")
    generate_btn = gr.Button("Generate")
    clear_btn.click(fn=clear_article, outputs=article_input)
    file_upload.change(fn=process_file, inputs=file_upload, outputs=article_input)
    generate_btn.click(fn=generate_response, inputs=[article_input, user_message, mode], outputs=output)

# Launch the app
demo.launch()